In [ ]:
#####################################################################
# Mandatory : Run this line and restart the notebook kernel right after
#####################################################################
!pip install torch==1.11 transformers

In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
# cd /content
# rm -rf deepqa
# git clone -b model https://github.com/PaulBeuran/deepqa.git

In [ ]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
#%cd deepqa/notebooks/

In [1]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8937k  100 8937k    0     0  3438k      0  0:00:02  0:00:02 --:--:-- 3438k


Archive:  squad1.1.zip
  inflating: data/dev-v1.1.json      
  inflating: data/train-v1.1.json    


In [4]:
import sys
import os
import pathlib

import json

import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import preprocessing, tokenizer, model, wrapper, loss, metrics, utils

2022-07-29 14:11:54.666575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 14:11:54.666669: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [6]:
hf_language_model_path = "distilbert-base-uncased"
context_max_length = 512
query_max_length = 64

train_contexts_token_ids, train_queries_token_ids, train_answers_token_ranges = (
    preprocessing.tokenize_squad_11_data(
        train_data, tokenizer.HFAutoTokenizer(hf_language_model_path), 512, 64
    )
)

dev_contexts_token_ids, dev_queries_token_ids, dev_answers_token_ranges = (
    preprocessing.tokenize_squad_11_data(
        dev_data, tokenizer.HFAutoTokenizer(hf_language_model_path), 512, 64
    )
)

In [7]:
train_dataset = utils.QADataset(train_contexts_token_ids, 
                                train_queries_token_ids, 
                                train_answers_token_ranges)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=8, 
                                           shuffle=False)

dev_dataset = utils.QADataset(dev_contexts_token_ids, 
                              dev_queries_token_ids, 
                              dev_answers_token_ranges)
train_loader = torch.utils.data.DataLoader(dev_dataset, 
                                           batch_size=8, 
                                           shuffle=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
word_encoder = (transformers.AutoModel.from_pretrained(hf_language_model_path)
                                        .embeddings
                                        .word_embeddings)
char_encoder = None
contextual_embedding_size = 256
bidaf = model.BiDAF(word_encoder, char_encoder, contextual_embedding_size)

In [ ]:
bidaf_trainer = model.QATrainWrapper(bidaf)
bidaf_trainer.train(train_loader, 
                    epochs=16, 
                    loss=loss.bi_cross_entropy)